## Notebook to prototype Chip Counting Algorithm
Given image, output the number of chips present

Potential Problems:
* Overlapping Chips
* Different Coloured Chips

In [1]:
import cv2 as cv
import os
import numpy as np
import pandas as pd
from helper_functions import read_imgs

In [28]:
img_dict = read_imgs(r"C:\Users\Asus\Downloads\nus-sds-dsc2021\train_data\train_images")

In [14]:
df = pd.read_csv(r"C:\Users\Asus\Downloads\nus-sds-dsc2021\train_data\train_labels.csv")
df

,img_id,object_count_gt,handwritten_count_gt,anomalies_bbox_gt
0,1001,80,80,[]
1,1002,6,6,[]
2,1003,51,52,"[('black_spot', [67, 44, 27, 16])]"
3,1004,24,24,[]
4,1005,77,77,[]
...,...,...,...,...
111,1112,200,-1,[]
112,1113,200,-1,[]
113,1114,198,-1,[]
114,1115,200,-1,[]


In [71]:
pred = {}
for k in img_dict.keys():
    count = 0
    img = cv.resize(img_dict[k].copy(), (round(440*0.9), round(610*0.9)))
    lab= cv.cvtColor(img, cv.COLOR_BGR2LAB)
    l, a, b = cv.split(lab)

    ret, th = cv.threshold(b, 148, 255, cv.THRESH_BINARY);
#     ret, th = cv.threshold(b, 127, 255, cv.THRESH_OTSU);
    
    contours, hierarchy = cv.findContours(th, mode=cv.RETR_TREE, method=cv.CHAIN_APPROX_SIMPLE)
    mask = np.zeros_like(img)
    for cnt in contours:
        approx = cv.approxPolyDP(cnt, 0.01 * cv.arcLength(cnt, True), True)
        area = cv.contourArea(approx)
        if 150 < area:
            cv.drawContours(mask, [approx], -1, (0, 255, 0), 1)
            count += area//100
        elif 50 < area < 150:
            cv.drawContours(mask, [approx], -1, (0, 255, 0), 1)
            count += 1
        else:
            cv.drawContours(mask, [approx], -1, (255, 0, 0), 1)
    pred[k] = int(count)


#     th = cv.cvtColor(th, cv.COLOR_GRAY2BGR)
#     b = cv.cvtColor(b, cv.COLOR_GRAY2BGR)
    
#     cv.imshow("window", cv.hconcat([th, img, b]))
#     cv.waitKey(0)
#     cv.destroyAllWindows()
    

In [69]:
error = 0
for i in range(len(df)):
    k = df.iloc[i]["img_id"]
    error += abs(pred[k] - df.iloc[i]["object_count_gt"])
error

3944

In [70]:
for i in range(len(df)):
    k = df.iloc[i]["img_id"]
    print(k, pred[k], df.iloc[i]["object_count_gt"], abs(pred[k] - df.iloc[i]["object_count_gt"]))

1001 148 80 68
1002 12 6 6
1003 98 51 47
1004 23 24 1
1005 76 77 1
1006 47 47 0
1007 67 67 0
1008 30 30 0
1009 49 45 4
1010 8 8 0
1011 32 32 0
1012 9 9 0
1013 46 48 2
1014 6 6 0
1015 124 124 0
1016 71 71 0
1017 83 83 0
1018 29 29 0
1019 153 139 14
1020 30 30 0
1021 158 153 5
1022 34 35 1
1023 58 58 0
1024 6 7 1
1025 235 200 35
1026 251 200 51
1027 130 128 2
1028 68 68 0
1029 229 200 29
1030 222 200 22
1031 122 121 1
1032 33 33 0
1033 407 375 32
1034 411 375 36
1035 377 375 2
1036 262 214 48
1037 375 341 34
1038 275 251 24
1039 393 300 93
1040 499 375 124
1041 380 375 5
1042 270 265 5
1043 102 100 2
1044 196 177 19
1045 192 126 66
1046 375 375 0
1047 393 375 18
1048 98 98 0
1049 103 106 3
1050 289 285 4
1051 414 375 39
1052 151 149 2
1053 490 375 115
1054 248 188 60
1055 307 256 51
1056 76 61 15
1057 424 375 49
1058 105 76 29
1059 243 146 97
1060 103 84 19
1061 5 5 0
1062 99 84 15
1063 2 3 1
1064 24 24 0
1065 21 48 27
1066 20 20 0
1067 53 49 4
1068 768 480 288
1069 138 113 25
1070 144 9

In [50]:
# 32 for merge issue
img = cv.resize(img_dict[1112].copy(), (round(440*0.9), round(610*0.9)))
count = 0

lab= cv.cvtColor(img, cv.COLOR_BGR2LAB)
l, a, b = cv.split(lab)

ret, th = cv.threshold(b, 135, 255, cv.THRESH_BINARY);
# ret, th = cv.threshold(b, 0, 255, cv.THRESH_OTSU);

contours, hierarchy = cv.findContours(th, mode=cv.RETR_TREE, method=cv.CHAIN_APPROX_SIMPLE)
mask = np.zeros_like(img)
# x = cv.drawContours(mask, contours, -1, (0, 255, 0), 1)
for cnt in contours:
    approx = cv.approxPolyDP(cnt, 0.01 * cv.arcLength(cnt, True), True)
    area = cv.contourArea(approx)
    if 150 < area:
        cv.drawContours(mask, [approx], -1, (0, 255, 0), 1)
        count += area//100
        print(area, area//100)
    elif 50 < area < 150:
        cv.drawContours(mask, [approx], -1, (0, 0, 255), 1)
        count += 1
    else:
        cv.drawContours(mask, [approx], -1, (255, 0, 0), 1)

print(count)

154.5 1.0
1269.0 12.0
395.0 3.0
541.5 5.0
8804.0 88.0
9080.5 90.0
6535.5 65.0
7253.0 72.0
4916.0 49.0
387.0


In [51]:
th = cv.cvtColor(th, cv.COLOR_GRAY2BGR)
b = cv.cvtColor(b, cv.COLOR_GRAY2BGR)
cv.imshow("window", cv.hconcat([th, img, b]))
# cv.imshow("window", mask)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
# img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# blur = cv.bilateralFilter(img,30,40,40)

# th = cv.adaptiveThreshold(blur, maxValue=255, blockSize=11, adaptiveMethod=cv.ADAPTIVE_THRESH_GAUSSIAN_C, thresholdType=cv.THRESH_BINARY, C=2)
# img_morph = cv.morphologyEx(th, cv.MORPH_CLOSE, (5,5), iterations=1)

# # dst = img_morph

# contours, hierarchy = cv.findContours(img_morph, mode=cv.RETR_TREE, method=cv.CHAIN_APPROX_SIMPLE)
# mask = np.zeros_like(img_list[0].copy())
# # cv.drawContours(mask, contours, -1, (0, 255, 0), 1)
# for cnt in contours:
#     approx = cv.approxPolyDP(cnt, 0.01 * cv.arcLength(cnt, True), True)
#     if (len(approx) != 4):
#         continue
#     cv.drawContours(mask, [approx], -1, (0, 255, 0), 1)

# dst = cv.Canny(blur, threshold1=5, threshold2=50, edges=None, apertureSize=5)
# lines = cv.HoughLines(dst, rho=1, theta=np.pi/180, threshold=150, lines=None, srn=0, stn=0,)# min_theta=0, max_theta=np.pi/2+0.2)
# lines = cv.HoughLinesP(dst, rho=1, theta=np.pi/180, threshold=50, lines=None, minLineLength=100, maxLineGap=100)

# if lines is not None:
#     for i in range(0, len(lines)):
#         line = lines[i][0]
#         cv.line(img, (line[0], line[1]), (line[2], line[3]), (0,0,255),1)

# for rho,theta in lines[0]:
#     a = np.cos(theta)
#     b = np.sin(theta)
#     x0 = a*rho
#     y0 = b*rho
#     x1 = int(x0 + 1000*(-b))
#     y1 = int(y0 + 1000*(a))
#     x2 = int(x0 - 1000*(-b))
#     y2 = int(y0 - 1000*(a))
#     cv.line(img,(x1,y1),(x2,y2),(0,0,255),3)

# len(lines)